# Camera running scripts 
Each script below should be saved to an individual .py file (except the bash .sh script). All the files should be kept in the same folder on the Pi.  

## Before running the Run_Camera.sh file
Before running the camera make sure an empty folder called "Data" is on the hard drive, and an empty "check.txt" text file is also within it. This is where the video and audio files will be saved.

## Run_Camera.sh 
This bash script connects the camera run scripts together and is used as the main file. Save as "Run_Camera.sh"

In [ ]:
sleep 120 #This allows time for the harddrive to attach on boot before filming starts. 
cd /home/pi/Camera_Files #Path to main folder where all scripts .py are stored
python3 ./Calculate_Iterations.py 
sleep 10
python3 ./Check_harddrive.py 
python3 ./Audio_Video_Capture.py #This runs the camera 
python3 ./Check_reset.py

## Calculate_Iterations.py
This script calculates how many 30 minute (or user specified) filming chunks to run when the camera is turned on. The user can specify at which time point they would like the camera to stop filming. If the camera is turned on after filming is due to finish, this script ensures the camera does not film. 

In [ ]:
##############################################
# Calculate number of iterations
##############################################
from datetime import datetime, timedelta, date, time

## User specified info
cut = time(17,30,0) #Cut off time to stop recording e.g. 5:30pm 
length = 30 #Recording length in mins

##################
secduration = length*60 #Recording length in seconds

date=date.today()

compare = datetime.combine(date, cut)

d = compare-datetime.now()
days =d.days
seconds = d.seconds
minutes = seconds/60
chunks = minutes/length
chunks=round(chunks,0)

if days == 0:
    Iterations = int(chunks)

elif days <0:
    Iterations = 0

## Check_harddrive.py 
This script checks whether the hard drive is writable. If not, video is saved to the microsd (failsafe). For this script to work an empty text file called "check.txt" should be added to the Data folder on the hard drive. 

In [ ]:
##############################################
# Check if hard drive is writeable
##############################################
f=open('/media/pi/Cam7/Video_Audio/Data/check.txt', 'a') #replace filepath with users own filepath
ok=f.writable()

if ok == True:
    Store = '/media/pi/Cam7/Video_Audio/Data/' #replace filepath with users own filepath
else:
    #Storage location on Pi's microsd - backup incase there is an issue with harddrive
    Store = '/home/pi/Data/' 
print(Store)
    

## Audio_Video_Capture.py
This script runs the video and audio capture. It has been modified from Mouy et al. (2020) and settings can be edited by the user. The video and audio are collected in 30 minute sequential chunks (this can be set by the user in Calculate_Iterations.py). We recommend filming in shorter chunks (e.g. 30 minutes) to help with processing both by the Raspberry Pi and the user. 

Video and audio are captured separately and can be joined together using FFMPEG if necessary - when calibrating in situ (i.e. moving the checkerboard) make sure to temporally calibrate the video and audio using a clapperboard equivalent (e.g. banging on the checkerboard in view of both cameras). 

Please note this file will not work with the buster operating system (please see "Camera-running-files" folder for replacement file) or the Arducam 16MP Camera Module. 

Mouy, X., Black, M., Cox, K., Qualley, J., Mireault, C., Dosso, S. and Juanes, F., 2020. FishCam: A low-cost open source autonomous camera for aquatic research. HardwareX, 8, p.e00110.

In [ ]:
##############################################
# Capture video and audio
##############################################
import picamera
import subprocess
import os
import time as t
import signal
from multiprocessing import Process
from threading import Thread
import argparse
from datetime import datetime
from Calculate_Iterations import Iterations, secduration
import logging
from Check_harddrive import Store


def initVideoSettings(): #VideoSettings from Mouy et al. (2020)
    videoSettings = {
        'resolution': (1920,1080),
        'frameRate': 30,      # frames per second
        'quality': 18,        # 1 = highest quality (but largest file size), 30 = lowest quality 
        'format': 'h264',     # 'h264' works better than mpeg but needs to be converted after (see code below)
        'exposure': 'night',  # 'auto', 'night','backlight'- for underwater use "night"
        'AWB': 'auto',        # 'auto', 'cloudy', 'sunlight'
        'sharpness': 0,       # value -100 to 100, auto: 0 (integer)
        'contrast': 0,        # value -100 to 100, auto: 0 (integer)
        'brightness': 50,     # value 0 to 100, auto: 0 (integer)
        'saturation': 0,      # value -100 to 100, auto: 0 (integer)
        'ISO': 400,           # low sensitivity: 100, high sensitivity: 400,800, auto: 0 
        'vflip': False,
        'output':Store,       #Filepath automatically added from Check_harddrive.py
        'CamID':'Cam7'        #Add name of harrdrive e.g. Cam7 - this will add the harddrive name to the filepath 
        }
    return videoSettings

    
def record_video(videoSettings=0):
    if videoSettings == 0:
        videoSettings = initVideoSettings()
    with picamera.PiCamera() as camera:
        camera.framerate = videoSettings['frameRate']
        camera.resolution = videoSettings['resolution']
        camera.exposure_mode = videoSettings['exposure']
        camera.awb_mode = videoSettings['AWB']
        camera.vflip = videoSettings['vflip']
        camera.sharpness = videoSettings['sharpness']
        camera.contrast = videoSettings['contrast']
        camera.brightness = videoSettings['brightness']
        camera.saturation = videoSettings['saturation']
        camera.iso = videoSettings['ISO']
        camera.start_recording(videoSettings['output']+videoSettings['CamID']+
                               '_' +str(i)+'_Video_'+ datetime.now().strftime("%Y-%m-%d_%H-%M-%S-%f")+
                               '.h264', format=videoSettings['format'], quality=videoSettings['quality'])
        camera.wait_recording(secduration)
        camera.stop_recording()

def record_audio(videoSettings=0):
    if videoSettings == 0:
        videoSettings = initVideoSettings()
    dur=videoSettings['duration']
    dur=f'--duration={dur} '
    date=str(datetime.now())
    name= videoSettings['output']+videoSettings['CamID']+'_'+str(i)+'_Audio_'+ 
        datetime.now().strftime("%Y-%m-%d_%H-%M-%S-%f")
    record = 'arecord -D dmic_sv -c2 -r 48000 -f S32_LE -t wav -V mono -v ' + dur + name+'.wav' 
    rec_proc=subprocess.Popen(record, shell=True)                   

def warmup(videoSettings = 0):
    if videoSettings == 0:
        videoSettings = initVideoSettings()
    with picamera.PiCamera() as camera:
        camera.framerate = videoSettings['frameRate']
        camera.resolution = videoSettings['resolution']
        camera.exposure_mode = videoSettings['exposure']
        camera.awb_mode = videoSettings['AWB']
        camera.vflip = videoSettings['vflip']
        camera.sharpness = videoSettings['sharpness']
        camera.contrast = videoSettings['contrast']
        camera.brightness = videoSettings['brightness']
        camera.saturation = videoSettings['saturation']
        camera.iso = videoSettings['ISO']
        print('Warming up camera')
        print(datetime.now())
        t.sleep(2)

log_path=os.path.join(Store,'error.log')
logging.basicConfig(filename=log_path, level = logging.ERROR)
try:
    warmup()
    for i in range(Iterations):
        v=Thread(target=record_video)
        a=Thread(target=record_audio)
        v.start()
        a.start()
            
        v.join()
        a.join()
except Exception as e:
    logging.exception(str(e))

## Check_reset.py
This script checks whether the camera has finished filming when it should have, or whether the Audio_Video_Capture.py script has finished prematurely (for an unforeseen reason). If the script has finished prematurely it reboots the Pi which restarts the filming process. 

In [ ]:
from Calculate_Iterations import Iterations, secduration, cut, compare
from datetime import datetime, timedelta, date, time
import os

now = datetime.now()
hr = now.hour

compare_hr=compare.hour

if hr < compare_hr: 
    print("restart")
    os.system("sudo reboot")

## Convert_h264_video.py
Video is collected in h264 as it is easier to process by the Pi. The video can then be converted (at a later date after filming) using the below code. It converts any video within a folder with "Data" in the name. After filming folders should therefore be named something like: Cam7_Data_SiteName_Date. This script creates a duplicate folder (named Cam7_Converted_SiteName_Date) with converted video files (with duplicate names but .mp4 format). 

In [ ]:
## Prior to use please install gpac using:
sudo apt-get install -y gpac

In [ ]:
from pathlib import Path
import os
import fnmatch
import subprocess

#Add main file path here, within this path we have the folder "Cam7_Data_SiteName_Date"
files=Path('/media/pi/Cam7/Video_Audio/') 
video_fps = 30 # Frame per second

for folder in files.iterdir():
    name=os.path.splitext(folder.name)[0]
    if 'Data' in name:
        changed_name= name.replace('Data','')
        convert_name_new='Convert_'+changed_name
        convert_name_path=os.path.join(files,convert_name_new)
        exist_check=os.path.exists(convert_name_path)
        if not exist_check:
            os.makedirs(convert_name_path)
        vids=Path(folder)
        for file in vids.glob('*.h264'):
            vid_name=os.path.splitext(file.name)[0]
            in_dir=os.path.join(files, folder.name)
            input_video= in_dir+'/' +vid_name +'.h264'
            output_video=convert_name_path+'/' +vid_name +'.mp4'
            exist=os.path.exists(output_video)
            if not exist:    
                subprocess.call(["MP4Box", "-fps", str(video_fps), "-add", input_video, output_video])